# financial statements testing grounds


In [1]:
import yfinance as yf
import pandas as pd

In [2]:
import requests
import time


api_key = "TULH49F5XQMU8MQ5"
base_url = "https://www.alphavantage.co/query"

def get_fin_stmt(symbol, function):
    url = base_url + "?function=" + function + "&symbol=" + symbol + "&apikey=" + api_key
    response = requests.get(url)
    time.sleep(16)
    if response.status_code != 200:
        print("Error, cannot retrieve" + function + "for " + symbol)
        return None
    data = response.json()
    if "annualReports" not in data:
        print("Error: No " + function + " data found for " + symbol)
        return None
#     print(data)
    statement_data = data["annualReports"]
    return_dct = {}
    
    return statement_data
statement_dct = {}


# retrieving data using above function in compressed format
statements = {}
stocks = ["ENPH", "IBM", "MBUU", "MANH", "TREX", "LSCC"]

statement_types =  ["INCOME_STATEMENT", "BALANCE_SHEET", "CASH_FLOW","EARNINGS"]
for stock in stocks:
    statements[stock] = {}
    for statement_type in statement_types:
        statement_key = statement_type.split("_")[0]
        statements[stock][statement_key] = {}
        try:
            fin_statement = get_fin_stmt(stock, statement_type)
            for annual_data in fin_statement:
                date = annual_data["fiscalDateEnding"]

                statements[stock][statement_key][date] = annual_data
        except:
            print("Error retrieving " + statement_type + " for " + stock)


Error: No EARNINGS data found for ENPH
Error retrieving EARNINGS for ENPH
Error: No EARNINGS data found for IBM
Error retrieving EARNINGS for IBM
Error: No EARNINGS data found for MBUU
Error retrieving EARNINGS for MBUU
Error: No EARNINGS data found for MANH
Error retrieving EARNINGS for MANH
Error: No EARNINGS data found for TREX
Error retrieving EARNINGS for TREX
Error: No EARNINGS data found for LSCC
Error retrieving EARNINGS for LSCC


In [3]:
plot_df = pd.DataFrame(statements["ENPH"]["INCOME"]).T
plot_df

,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,...,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome
2022-12-31,2022-12-31,USD,974595000,2330853000,1356258000,1356258000,448261000,355104000,168846000,526334000,...,27700000,24696000,452048000,54686000,9438000,397362000,388500000,461486000,486182000,397362000
2021-12-31,2021-12-31,USD,554422000,1382049000,827627000,827627000,215832000,233064000,105526000,338590000,...,16700000,9500000,120928000,-24521000,101649000,145449000,142995000,166080000,175580000,145449000
2020-12-31,2020-12-31,USD,345981000,774425000,428444000,428444000,186439000,103621000,55921000,159542000,...,9700000,5092000,119410000,-14585000,21001000,133995000,135352000,140411000,145503000,133995000
2019-12-31,2019-12-31,USD,221245000,624333000,403088000,403088000,102729000,75536000,40381000,118516000,...,7300000,4727000,90114000,-71034000,17843000,161148000,159483000,99805000,104532000,161148000
2018-12-31,2018-12-31,USD,94445000,316159000,221714000,221714000,1596000,56133000,32587000,92849000,...,8300000,1400000,-10229000,1398000,9635000,-11627000,-10229000,464000,1864000,-11627000


In [4]:
plot_df.keys()

Index(['fiscalDateEnding', 'reportedCurrency', 'grossProfit', 'totalRevenue',
       'costOfRevenue', 'costofGoodsAndServicesSold', 'operatingIncome',
       'sellingGeneralAndAdministrative', 'researchAndDevelopment',
       'operatingExpenses', 'investmentIncomeNet', 'netInterestIncome',
       'interestIncome', 'interestExpense', 'nonInterestIncome',
       'otherNonOperatingIncome', 'depreciation',
       'depreciationAndAmortization', 'incomeBeforeTax', 'incomeTaxExpense',
       'interestAndDebtExpense', 'netIncomeFromContinuingOperations',
       'comprehensiveIncomeNetOfTax', 'ebit', 'ebitda', 'netIncome'],
      dtype='object')

In [5]:
def calc_ROIC(EBIT, Taxes, Dividends, Debt, Equity, Cash_Equiv):
    try:
        NOPAT = (int(EBIT) - int(Taxes)) - int(Dividends)
        Invested_Capital = int(Debt) + int(Equity) - int(Cash_Equiv)
        ROIC = round(NOPAT / Invested_Capital, 2)
    except:
        NOPAT = (int(EBIT) - int(Taxes))
        Invested_Capital = int(Debt) + int(Equity) - int(Cash_Equiv)
        ROIC = round(NOPAT / Invested_Capital, 2)
    return ROIC
    

In [6]:
def calc_EPS(Net_Income, Dividends, Shares):
    try:
        EPS = round((int(Net_Income) - int(Dividends)) / int(Shares),2)
    except:
        EPS = round((int(Net_Income)) / int(Shares),2)
    return EPS

In [7]:
# Core of valuation, subject to change suddenly over new valuation 
# methodologies. Hoping to have projected EPS based on growth rates and 
# diminishing linear projection for EPS and also ROIC numbers. Assigned
# multiple will be based on growth rates & ROIC in some way

def EPS_Valuation(EPS, ROIC):
    Multiple = (8.5 + 60*(ROIC))
    Price = Multiple * EPS
    return Price

In [8]:
#calcula
#calc_ROIC(statements["ENPH"]["INCOME"]["2022-12-31"]["ebit"], )
int(statements["ENPH"]["INCOME"]["2022-12-31"]["ebit"]) - int(statements[
    "ENPH"]["INCOME"]["2022-12-31"]["incomeTaxExpense"])


406800000

In [9]:
statements["ENPH"]["INCOME"]["2022-12-31"].keys()

dict_keys(['fiscalDateEnding', 'reportedCurrency', 'grossProfit', 'totalRevenue', 'costOfRevenue', 'costofGoodsAndServicesSold', 'operatingIncome', 'sellingGeneralAndAdministrative', 'researchAndDevelopment', 'operatingExpenses', 'investmentIncomeNet', 'netInterestIncome', 'interestIncome', 'interestExpense', 'nonInterestIncome', 'otherNonOperatingIncome', 'depreciation', 'depreciationAndAmortization', 'incomeBeforeTax', 'incomeTaxExpense', 'interestAndDebtExpense', 'netIncomeFromContinuingOperations', 'comprehensiveIncomeNetOfTax', 'ebit', 'ebitda', 'netIncome'])

In [10]:
statements["ENPH"]["BALANCE"]["2022-12-31"].keys()

dict_keys(['fiscalDateEnding', 'reportedCurrency', 'totalAssets', 'totalCurrentAssets', 'cashAndCashEquivalentsAtCarryingValue', 'cashAndShortTermInvestments', 'inventory', 'currentNetReceivables', 'totalNonCurrentAssets', 'propertyPlantEquipment', 'accumulatedDepreciationAmortizationPPE', 'intangibleAssets', 'intangibleAssetsExcludingGoodwill', 'goodwill', 'investments', 'longTermInvestments', 'shortTermInvestments', 'otherCurrentAssets', 'otherNonCurrentAssets', 'totalLiabilities', 'totalCurrentLiabilities', 'currentAccountsPayable', 'deferredRevenue', 'currentDebt', 'shortTermDebt', 'totalNonCurrentLiabilities', 'capitalLeaseObligations', 'longTermDebt', 'currentLongTermDebt', 'longTermDebtNoncurrent', 'shortLongTermDebtTotal', 'otherCurrentLiabilities', 'otherNonCurrentLiabilities', 'totalShareholderEquity', 'treasuryStock', 'retainedEarnings', 'commonStock', 'commonStockSharesOutstanding'])

In [11]:
statements["ENPH"]["CASH"]["2022-12-31"].keys()

dict_keys(['fiscalDateEnding', 'reportedCurrency', 'operatingCashflow', 'paymentsForOperatingActivities', 'proceedsFromOperatingActivities', 'changeInOperatingLiabilities', 'changeInOperatingAssets', 'depreciationDepletionAndAmortization', 'capitalExpenditures', 'changeInReceivables', 'changeInInventory', 'profitLoss', 'cashflowFromInvestment', 'cashflowFromFinancing', 'proceedsFromRepaymentsOfShortTermDebt', 'paymentsForRepurchaseOfCommonStock', 'paymentsForRepurchaseOfEquity', 'paymentsForRepurchaseOfPreferredStock', 'dividendPayout', 'dividendPayoutCommonStock', 'dividendPayoutPreferredStock', 'proceedsFromIssuanceOfCommonStock', 'proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet', 'proceedsFromIssuanceOfPreferredStock', 'proceedsFromRepurchaseOfEquity', 'proceedsFromSaleOfTreasuryStock', 'changeInCashAndCashEquivalents', 'changeInExchangeRate', 'netIncome'])

In [12]:
statements["ENPH"]["CASH"]["2022-12-31"]["dividendPayout"]

'None'

In [13]:
import datetime

In [14]:
#for date in statements["ENPH"]["BALANCE"]:
#    format = "%Y-%m-%d"
    
    
#    print(statements["ENPH"]["BALANCE"][date])

In [15]:
import numpy as np

In [16]:
# here we calculate a core component of our valuation strategy, ROIC
# We believe that ROIC is a good indicator of return to shareholder capital,
# and in our thesis shows what the return on the money generated by the 
# business could potentially be. This will be used as a guage for our assigned
# multiple for businesses. I am also adding a cash&Equivalents portion
# to artificially inflate businesses with higher cash stores. This
# is meant to give a higher multiple to cash-neutral or cash-holding businesses
# due to decreased liquidity risk. Might tweak this later.

# right now we are removing dividends from the EPS calculation since I will be adding dividend yields into the 
# valuation later on if I have time. Note: underestimating multiples for all companies, but strongly on dividend companies


for stock in statements:
    statements[stock]["VALUATION"] = {}
    for date in statements[stock]["BALANCE"]:
            statements[stock]["VALUATION"][date] = {}
            new = statements[stock]["VALUATION"][date]
            old = statements[stock]
            new["ROIC"] = calc_ROIC(EBIT = old["INCOME"][date]["ebit"], 
                                    Taxes = old["INCOME"][date]["incomeTaxExpense"],
                                    Dividends = old["CASH"][date]["dividendPayout"],
                                    Debt = old["BALANCE"][date]["totalLiabilities"],
                                   Equity = old["BALANCE"][date]["totalShareholderEquity"],
                                   Cash_Equiv = old["BALANCE"][date]["cashAndShortTermInvestments"])
            new["EPS"] = calc_EPS(Net_Income = old["INCOME"][date]["netIncome"], 
                                   Dividends = old["CASH"][date]["dividendPayout"],
                                  Shares = old["BALANCE"][date]["commonStockSharesOutstanding"])
            statements[stock]["VALUATION"][date]["EPS_Valuation"] = EPS_Valuation(EPS = new["EPS"],
                                                                                 ROIC = new["ROIC"])

In [20]:
#for stock in statements:
#    statements[stock]["VALUATION"]["ROIC_5-year"] = new["ROIC"].mean()
 #   statements[stock]["VALUATION"]["EPS_5-year"] = new["EPS"].mean()
        #print(stock, date, statements[stock]["BALANCE"][date]["commonStockSharesOutstanding"])
statements["MBUU"]["VALUATION"]

{'2022-06-30': {'ROIC': 0.22, 'EPS': 7.57, 'EPS_Valuation': 164.269},
 '2021-06-30': {'ROIC': 0.16, 'EPS': 5.19, 'EPS_Valuation': 93.93900000000002},
 '2020-06-30': {'ROIC': 0.15, 'EPS': 2.9, 'EPS_Valuation': 50.75},
 '2019-06-30': {'ROIC': 0.17,
  'EPS': 3.07,
  'EPS_Valuation': 57.409000000000006},
 '2018-06-30': {'ROIC': 0.11, 'EPS': 1.26, 'EPS_Valuation': 19.026}}

In [18]:
for stock in statements:
    for statement_type in statements[stock]:
        for date in statements[stock][statement_type]:
            

IndentationError: expected an indented block (3880963370.py, line 4)

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
for date in statements["ENPH"]["VALUATION"]:
    print(statements["ENPH"]["VALUATION"][date]["ROIC"])

In [ ]:
#data_dct = {}
#for json_dct in ENPH_IS:
#     date = json_dct["fiscalDateEnding"]
#    data_dct[date] = json_dct
#
#pd.DataFrame(data_dct).T

In [ ]:
data

In [ ]:
type(data)

import requests

# API key for Alpha Vantage
api_key = 'YOUR_API_KEY'

# Base URL for Alpha Vantage API
base_url = 'https://www.alphavantage.co/query'

# Ticker symbol for AAPL
symbol = 'AAPL'

# Initialize the data dictionary
data = {}

# Function to retrieve financial statement data for a given symbol and statement type
def get_financial_statement(symbol, statement_type):
    # Build the URL for the API call
    url = base_url + '?function=' + statement_type + '&symbol=' + symbol + '&apikey=' + api_key

    # Make the API call
    response = requests.get(url)

    # Check if the API call was successful
    if response.status_code != 200:
        print("Error: Unable to retrieve " + statement_type + " for " + symbol)
        return None

    # Extract the financial statement data from the API response
    data = response.json()
    statement_data = data['annualReports']

    return statement_data

# Retrieve and store the income statement for AAPL
income_statement = get_financial_statement(symbol, 'INCOME_STATEMENT')
data[symbol] = {'Income Statement': income_statement}

# Retrieve and store the balance sheet for AAPL
balance_sheet = get_financial_statement(symbol, 'BALANCE_SHEET')
data[symbol]['Balance Sheet'] = balance_sheet

# Retrieve and store the cash flow statement for AAPL
cash_flow = get_financial_statement(symbol, 'CASH_FLOW')
data[symbol]['Cash Flow'] = cash_flow

# Print the retrieved data
print(data[symbol]['Income Statement'])
print(data[symbol]['Balance Sheet'])
print(data[symbol]['Cash Flow'])
